In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import datetime

/Users/sdegryze/miniconda3/envs/semseg/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.4.0


/Users/sdegryze/miniconda3/envs/semseg/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: No GPU found. Please use a GPU to train your neural network.
  import sys


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)


Tests Passed


In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """

    # By scaling layers 3 and 4, the model learns much better (i.e. higher accuracy/IoU)
    # See https://discussions.udacity.com/t/here-is-some-advice-and-clarifications-about-the-semantic-segmentation-project/403100
    
    vgg_layer3_out_scaled = tf.multiply(vgg_layer3_out, 0.0001, name='pool3_out_scaled')
    vgg_layer4_out_scaled = tf.multiply(vgg_layer4_out, 0.01, name='pool4_out_scaled')

    # First run 1x1 convolutions to collapse the number of channels/filters into just num_classes (i.e., 2)
    # conv1_7 has 512 channels/filters
    # Note the way we are regularizing here. We still need to add these regularization terms to the final loss
    # Not sure if padding = 'same' is needed given that we're doing 1x1 convolutions
    
    conv1_7 = tf.layers.conv2d(vgg_layer7_out, num_classes, kernel_size=1, padding='same',
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                               kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    conv1_4 = tf.layers.conv2d(vgg_layer4_out_scaled, num_classes, kernel_size=1, padding='same',
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                               kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    conv1_3 = tf.layers.conv2d(vgg_layer3_out_scaled, num_classes, kernel_size=1, padding='same',
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                               kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    # strides of (2, 2) is what is doing the up-sampling here
    contrans1 = tf.layers.conv2d_transpose(conv1_7, num_classes, kernel_size=4, strides=(2, 2),
                                           padding='same',
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    contrans_add1 = tf.add(contrans1, conv1_4)
    
    contrans2 = tf.layers.conv2d_transpose(contrans_add1, num_classes, kernel_size=4, strides=(2, 2),
                                           padding='same',
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    contrans_add2 = tf.add(contrans2, conv1_3)
    
    contrans_output = tf.layers.conv2d_transpose(contrans_add2, num_classes, kernel_size=16, strides=(8, 8),
                                                 padding='same',
                                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    return contrans_output
tests.test_layers(layers)

Tests Passed


In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    
    # Convert to a 2D tensor where each row represents a pixel and each column a class.
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    # why softmax, since there only 2 labels. One could just do a logistic regression.
    # Likely, this is because a softmax is more general
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label),
                                       name='fcn_cross_entropy_loss')
    # add all of the regularization terms that were introduced through the kernel_regularizer arguments
    regularization_term = sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    total_loss = cross_entropy_loss + regularization_term
    with tf.name_scope('summaries'):
        tf.summary.scalar('total_loss', total_loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    return logits, train_op, total_loss
tests.test_optimize(optimize)

Tests Passed


In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, total_loss, input_image,
             correct_label, keep_prob, learning_rate, train_writer, merged, saver,
             learning_rate_val=0.001,
             keep_prob_val=0.5):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param total_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    
    overall_batch_nr = 0
    for epoch in epochs:
        batch_nr = 0
        loss_accumul = 0
        
        for image, label in get_batches_fn(batch_size):

            
            _ , loss, summary = sess.run([train_op, total_loss, merged],
                                feed_dict = {input_image: image,
                                             correct_label: label,
                                             learning_rate: learning_rate_val,
                                             keep_prob: keep_prob_val})
            loss_accumul += loss
            print("{} Epoch {}, batch {}, loss {}".format(str(datetime.datetime.now()), epoch + 1, batch_nr + 1, loss)) 
            
            batch_nr += 1
            overall_batch_nr += 1
            train_writer.add_summary(summary, overall_batch_nr)
        avg_loss = (loss_accumul / float(batch_nr))
        print("Epoch {}, average loss {}".format(epoch + 1, avg_loss))
        model_ckpt_name = "lr{}keep{}ep{}loss{}".format(str(learning_rate_val).replace(".", "_"),
                                                        str(keep_prob_val).replace(".", "_"),
                                                        epoch + 1,
                                                        str(avg_loss).replace(".", "_")) + ".ckpt"
        save_path = saver.save(sess, os.path.join("./models", model_ckpt_name))
        print("---------------------------------------")
            

In [7]:
def run(start_from=None):
    num_classes = 2
    image_shape = (160, 576)
    epochs = range(20)
    batch_size = 8
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    tf.reset_default_graph()
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        correct_label = tf.placeholder(tf.float32, shape=None, name='correct_label')
        learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate') 
        
        image_input, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        contrans_output = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        logits, train_op, total_loss = optimize(contrans_output, correct_label, learning_rate, num_classes)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter("./summaries/", sess.graph)
        saver = tf.train.Saver()
        
        if start_from is None:
            sess.run(tf.global_variables_initializer())
        else:
            saver.restore(sess, start_from)
            
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, total_loss, image_input,
                 correct_label, keep_prob, learning_rate, train_writer, merged, saver)

        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)

In [8]:
run("./models/lr0_001keep0_5ep20loss0_268072330952.ckpt")
#run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
INFO:tensorflow:Restoring parameters from ./models/lr0_001keep0_5ep20loss0_268072330952.ckpt
2018-01-08 21:38:15.035415 Epoch 1, batch 1, loss 0.23808833956718445
2018-01-08 21:38:40.491349 Epoch 1, batch 2, loss 0.27822378277778625
2018-01-08 21:39:07.640138 Epoch 1, batch 3, loss 0.2699750065803528
2018-01-08 21:39:35.203568 Epoch 1, batch 4, loss 0.2716173529624939
2018-01-08 21:40:04.411240 Epoch 1, batch 5, loss 0.3064248561859131
2018-01-08 21:40:32.943649 Epoch 1, batch 6, loss 0.30008578300476074
2018-01-08 21:40:59.362683 Epoch 1, batch 7, loss 0.2723190188407898
2018-01-08 21:41:25.751734 Epoch 1, batch 8, loss 0.2788800001144409
2018-01-08 21:41:52.485755 Epoch 1, batch 9, loss 0.29633527994155884
2018-01-08 21:42:19.213192 Epoch 1, batch 10, loss 0.29033660888671875
2018-01-08 21:42:46.131780 Epoch 1, batch 11, loss 0.31081679463386536
2018-01-08 21:43:13.234533 Epoch 1, batch 12, loss 

2018-01-08 22:24:52.704858 Epoch 4, batch 1, loss 0.2421664297580719
2018-01-08 22:25:18.252090 Epoch 4, batch 2, loss 0.3192604184150696
2018-01-08 22:25:43.933100 Epoch 4, batch 3, loss 0.2365300953388214
2018-01-08 22:26:09.309423 Epoch 4, batch 4, loss 0.2597091794013977
2018-01-08 22:26:34.689877 Epoch 4, batch 5, loss 0.2633725702762604
2018-01-08 22:26:59.448842 Epoch 4, batch 6, loss 0.27253687381744385
2018-01-08 22:27:23.545012 Epoch 4, batch 7, loss 0.25634390115737915
2018-01-08 22:27:47.850638 Epoch 4, batch 8, loss 0.25695252418518066
2018-01-08 22:28:12.173837 Epoch 4, batch 9, loss 0.26156240701675415
2018-01-08 22:28:36.735134 Epoch 4, batch 10, loss 0.25735989212989807
2018-01-08 22:29:01.108323 Epoch 4, batch 11, loss 0.2503288984298706
2018-01-08 22:29:25.200768 Epoch 4, batch 12, loss 0.2527956962585449
2018-01-08 22:29:49.401521 Epoch 4, batch 13, loss 0.2740842401981354
2018-01-08 22:30:13.838264 Epoch 4, batch 14, loss 0.24520167708396912
2018-01-08 22:30:38.068

2018-01-08 23:10:38.725381 Epoch 7, batch 4, loss 0.24588987231254578
2018-01-08 23:11:03.983525 Epoch 7, batch 5, loss 0.22296534478664398
2018-01-08 23:11:29.196842 Epoch 7, batch 6, loss 0.25086480379104614
2018-01-08 23:11:55.153094 Epoch 7, batch 7, loss 0.25413912534713745
2018-01-08 23:12:19.985018 Epoch 7, batch 8, loss 0.2099301964044571
2018-01-08 23:12:44.552336 Epoch 7, batch 9, loss 0.21539244055747986
2018-01-08 23:13:09.234241 Epoch 7, batch 10, loss 0.20049864053726196
2018-01-08 23:13:33.728617 Epoch 7, batch 11, loss 0.22134268283843994
2018-01-08 23:13:58.241681 Epoch 7, batch 12, loss 0.2261444628238678
2018-01-08 23:14:22.270178 Epoch 7, batch 13, loss 0.2031102031469345
2018-01-08 23:14:46.329690 Epoch 7, batch 14, loss 0.2340647429227829
2018-01-08 23:15:10.209492 Epoch 7, batch 15, loss 0.22096680104732513
2018-01-08 23:15:34.036824 Epoch 7, batch 16, loss 0.20681211352348328
2018-01-08 23:15:57.557718 Epoch 7, batch 17, loss 0.22453221678733826
2018-01-08 23:16

2018-01-08 23:54:40.866286 Epoch 10, batch 6, loss 0.32776233553886414
2018-01-08 23:55:04.469045 Epoch 10, batch 7, loss 0.29044729471206665
2018-01-08 23:55:28.037185 Epoch 10, batch 8, loss 0.2535068988800049
2018-01-08 23:55:51.808294 Epoch 10, batch 9, loss 0.23259443044662476
2018-01-08 23:56:15.267371 Epoch 10, batch 10, loss 0.24239715933799744
2018-01-08 23:56:38.701734 Epoch 10, batch 11, loss 0.33296075463294983
2018-01-08 23:57:02.227576 Epoch 10, batch 12, loss 0.2429637759923935
2018-01-08 23:57:26.017764 Epoch 10, batch 13, loss 0.26448339223861694
2018-01-08 23:57:49.546352 Epoch 10, batch 14, loss 0.2537609338760376
2018-01-08 23:58:13.175317 Epoch 10, batch 15, loss 0.2698958218097687
2018-01-08 23:58:36.690722 Epoch 10, batch 16, loss 0.26727354526519775
2018-01-08 23:59:00.415037 Epoch 10, batch 17, loss 0.2618742883205414
2018-01-08 23:59:24.060518 Epoch 10, batch 18, loss 0.225727379322052
2018-01-08 23:59:47.634568 Epoch 10, batch 19, loss 0.2682340145111084
2018

2018-01-09 00:39:38.945430 Epoch 13, batch 7, loss 0.1760045439004898
2018-01-09 00:40:03.774747 Epoch 13, batch 8, loss 0.1901189088821411
2018-01-09 00:40:28.114350 Epoch 13, batch 9, loss 0.19625595211982727
2018-01-09 00:40:52.260610 Epoch 13, batch 10, loss 0.19223091006278992
2018-01-09 00:41:16.858372 Epoch 13, batch 11, loss 0.18369540572166443
2018-01-09 00:41:41.232192 Epoch 13, batch 12, loss 0.23139598965644836
2018-01-09 00:42:05.524725 Epoch 13, batch 13, loss 0.18225736916065216
2018-01-09 00:42:29.884163 Epoch 13, batch 14, loss 0.20529824495315552
2018-01-09 00:42:54.589667 Epoch 13, batch 15, loss 0.1857418715953827
2018-01-09 00:43:19.005887 Epoch 13, batch 16, loss 0.1823497861623764
2018-01-09 00:43:43.250346 Epoch 13, batch 17, loss 0.1797570288181305
2018-01-09 00:44:07.970170 Epoch 13, batch 18, loss 0.2138483226299286
2018-01-09 00:44:32.234680 Epoch 13, batch 19, loss 0.20002198219299316
2018-01-09 00:44:56.560720 Epoch 13, batch 20, loss 0.1814439594745636
20

2018-01-09 01:23:55.400499 Epoch 16, batch 8, loss 0.15410611033439636
2018-01-09 01:24:18.862613 Epoch 16, batch 9, loss 0.1576811671257019
2018-01-09 01:24:42.558137 Epoch 16, batch 10, loss 0.1985192596912384
2018-01-09 01:25:06.106139 Epoch 16, batch 11, loss 0.17005081474781036
2018-01-09 01:25:29.575156 Epoch 16, batch 12, loss 0.1737866848707199
2018-01-09 01:25:53.243169 Epoch 16, batch 13, loss 0.18236877024173737
2018-01-09 01:26:16.915774 Epoch 16, batch 14, loss 0.16722515225410461
2018-01-09 01:26:40.505117 Epoch 16, batch 15, loss 0.19636599719524384
2018-01-09 01:27:04.063358 Epoch 16, batch 16, loss 0.13663263618946075
2018-01-09 01:27:27.830460 Epoch 16, batch 17, loss 0.1844235360622406
2018-01-09 01:27:51.373583 Epoch 16, batch 18, loss 0.20095625519752502
2018-01-09 01:28:15.181761 Epoch 16, batch 19, loss 0.1938750147819519
2018-01-09 01:28:38.712473 Epoch 16, batch 20, loss 0.19450891017913818
2018-01-09 01:29:02.350157 Epoch 16, batch 21, loss 0.1907988041639328


2018-01-09 02:07:48.110028 Epoch 19, batch 9, loss 0.15815109014511108
2018-01-09 02:08:12.793320 Epoch 19, batch 10, loss 0.13604313135147095
2018-01-09 02:08:36.969609 Epoch 19, batch 11, loss 0.12411093711853027
2018-01-09 02:09:01.271155 Epoch 19, batch 12, loss 0.17725157737731934
2018-01-09 02:09:25.434844 Epoch 19, batch 13, loss 0.17552638053894043
2018-01-09 02:09:49.928642 Epoch 19, batch 14, loss 0.1637367308139801
2018-01-09 02:10:14.381138 Epoch 19, batch 15, loss 0.15558147430419922
2018-01-09 02:10:38.510785 Epoch 19, batch 16, loss 0.15548449754714966
2018-01-09 02:11:02.744735 Epoch 19, batch 17, loss 0.1737828254699707
2018-01-09 02:11:27.247754 Epoch 19, batch 18, loss 0.1397133320569992
2018-01-09 02:11:51.533500 Epoch 19, batch 19, loss 0.1463049352169037
2018-01-09 02:12:15.800634 Epoch 19, batch 20, loss 0.1406955122947693
2018-01-09 02:12:39.993017 Epoch 19, batch 21, loss 0.13846012949943542
2018-01-09 02:13:04.541000 Epoch 19, batch 22, loss 0.1648245453834533

In [ ]:
def revive():
    num_classes = 2
    image_shape = (160, 576)
    epochs = range(3)
    batch_size = 64
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    tf.reset_default_graph()
    
    with tf.Session() as sess:
        # Path to vgg model
        model_path = os.path.join("./models", 'lr0_001keep0_5ep1loss1_26573096514.ckpt')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        
        imported_meta = tf.train.import_meta_graph(model_path + ".meta")
        imported_meta.restore(sess, model_path)
        
        graph = tf.get_default_graph()
        fcn_cross_entropy_loss_name = 'fcn_cross_entropy_loss:0'
        cross_entropy_loss = graph.get_tensor_by_name(fcn_cross_entropy_loss_name)
        vgg_input_tensor_name = 'image_input:0'
        vgg_keep_prob_tensor_name = 'keep_prob:0'

        input_image = graph.get_tensor_by_name(vgg_input_tensor_name)
        keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
        correct_label = graph.get_tensor_by_name('correct_label:0')
        
        batch_nr = 0
        batch_size = 8
        loss_accumul = 0
        for image, label in get_batches_fn(batch_size):
            loss = sess.run(cross_entropy_loss,
                                feed_dict = {input_image: image,
                                             correct_label: label,
                                             keep_prob: 1})
            loss_accumul += loss
            print("Batch {}, loss {}".format(batch_nr + 1, loss)) 
            batch_nr += 1
        avg_loss = (loss_accumul / float(batch_nr))
        print("Average loss  = {}".format(avg_loss))
revive()